In [17]:
using JuMP 
using Mosek

In [ ]:
#Script para el aprendizaje de grafos usando la metrica de Wasserstein, pero esta vez con la funcion de 
#costo ||.||1 que vuelve al problema lineal.

In [ ]:
# @param:weights : weights of the observed matrices. Sum of weights must be 1. and non-negative. the size is equal to the amount
#of observed graphs.
#@param: delta : size of the ball of the Wasserstein metric. delta > 0.
#@param: A : cluster form of the data. A entries must be 0 or 1. 
#@param: B : vector of matrices containing the observations of the graph B. If X,Y are matrices, an array of matrices
#is of the form Array[X,Y].
#@param: n : the dimension of the matrices.
#@param:N : number of observations.
#The set over which the is problem is optimized is:

In [ ]:
function learnLinearGraph(weights,delta,A,B,n)
    N = length(B)
    one =ones(n)
    m = Model(solver=MosekSolver())
    @variable(m,lambda>=0)
    @variable(m,S[1:N])
    @variable(m,A[1:n,1:n], Symmetric)
    #W debe ser simetrica? Creo que no.
    @variable(m, W[1:n,1:n],Symmetric)
    #a es una variable auxiliar para calcular la norma infinito de W
    @variable(m,a>=0)
    #Z es un conjunto de matrices que permite calcular la norma 1 de A-B[i].
    Z = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
    #O es un conjunto de matrices que juega el papel de Delta de la demostracion.
    O = [@variable(m, [1:n, 1:n],lowerbound = 0) for p in 1:N]
    #C es un conjunto de matrices que juega el papel de eta en la demostracion.
    E = [@variable(m, [1:n, 1:n],lowerbound = 0) for p in 1:N]
    
    #Constrains...
    #constrains for lambda already in the definition of the varible.
    #Constrains for A.
    @constraint(m,A.>=0)
    @constraint(m,A.<=1)
    #Constrains for Z.
    [@constraint(m, -Z[p][k,l] <= A[k,l]-B[p][k,l]) for p in 1:N for k in 1:n for l in 1:n]
    [@constraint(m,  A[k,l]-B[p][k,l]<=Z[p][k,l]) for p in 1:N for k in 1:n for l in 1:n]
    #Constrains for W.
    @constraint(m,a .>=W)
    @constraint(m,a .>=-W)
    @constraint(m,a<=lambda)
    
    #Constrains for O and E.
    [@constraint(m,2*A-one*one'-W+O[p]+E[p].==0) for p in 1:N]
    [@constraint(m, trace(O[p],one*one')<=S[p]-trace((2*A-one*one'-W)'*B[p])) for p in 1:N]
    
    #Objetivo
    @objective(m,Min,lambda*delta+ dot(weights,S+prod.(Z,n)))
    status = solve(m)
    return([getvalue(A),getvalue(S),getvalue(lambda),getvalue(W),getobjectivevalue(m)])
end

In [44]:
function learnLinearError(weights,delta,A,B,n)
    N = length(B)
    one =ones(n)
    m = Model(solver=MosekSolver())
    @variable(m,lambda>=0)
    @variable(m,S[1:N])
    #W debe ser simetrica? Creo que no.
    @variable(m, W[1:n,1:n],Symmetric)
    #a es una variable auxiliar para calcular la norma infinito de W
    @variable(m,a>=0)
    #Z es un conjunto de matrices que permite calcular la norma 1 de A-B[i].
    Z = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
    #O es un conjunto de matrices que juega el papel de Delta de la demostracion.
    O = [@variable(m, [1:n, 1:n],lowerbound = 0) for p in 1:N]
    #C es un conjunto de matrices que juega el papel de eta en la demostracion.
    E = [@variable(m, [1:n, 1:n],lowerbound = 0) for p in 1:N]
    
    #Constrains...
    #constrains for lambda already in the definition of the varible.
    #Constrains for Z.
    [@constraint(m, -Z[p][k,l] <= A[k,l]-B[p][k,l]) for p in 1:N for k in 1:n for l in 1:n]
    [@constraint(m,  A[k,l]-B[p][k,l]<=Z[p][k,l]) for p in 1:N for k in 1:n for l in 1:n]
    #Constrains for W. 
    @constraint(m,a .>=W)
    @constraint(m,a .>=-W)
    @constraint(m,a<=lambda)
    #Constrains for O and E.
    [@constraint(m,2*A-one*one'-W+O[p]+E[p].==0) for p in 1:N]
    [@constraint(m, trace(O[p],one*one')<=S[p]-trace((2*A-one*one'-W)'*B[p])) for p in 1:N]
    
    #Objetivo
    @objective(m,Min,lambda*delta+ dot(weights,S+prod.(Z,n)))
    status = solve(m)
    loss= getobjectivevalue(m)
    return(loss)
end

learnLinearError (generic function with 1 method)

In [15]:
matriz = [1 2;0 4]
matriz2 =[-1 -2; 0 -4]
#norm(vec(matriz),Inf)
matriz+matriz2 == 0

false

In [25]:
maximum(abs.(matriz2))

4

In [54]:
matriz = [-15 -2;-1 10]
m = Model(solver=MosekSolver())
E = [@variable(m, [1:3, 1:3]) for p in 1:2]
@constraint(m,E[1][1,1]==1)
@constraint(m,E[2][1,1]==-1)
@objective(m,Min,E[1][1,1]+E[2][1,1])
status = solve(m)
getobjectivevalue(m)
getvalue(E[2])

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 2               
  Cones                  : 0               
  Scalar variables       : 18              
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 0                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.01    
Optimizer terminated. Time: 0.01    


Interior-point solution summary
  Problem status  : PRIMAL_AND_DUAL_FEASIBLE
  Solution status : OPTIMAL
  Primal.  obj: 0.0000000000e+00    nrm: 1e+00    Viol.  con: 0e+00    var: 

3×3 Array{Float64,2}:
 -1.0  0.0  0.0
  0.0  0.0  0.0
  0.0  0.0  0.0

<h3>Funciones auxiliares<h3>

In [ ]:
#Aux functions
#def function prod computes the product 1^t*Z*t.
#@param: n : the dimension of the matrices.
function prod(Z,n)
    return(ones(n)'*Z*ones(n))
end
